# 04 - Evaluation & Verification

## Learning Goals

* Compute and interpret simple metrics (accuracy, confusion matrix).
* Produce reproducible **artifacts** and a machine-readable **receipt** for CI.
* Understand why verifiable outputs matter in a classroom or production pipeline.

## You Should Be Able To...

- Run model evaluation and interpret results
- Understand confusion matrices and accuracy metrics
- Generate verification receipts for ML pipelines
- Identify when models meet deployment criteria
- Reflect on the complete ML development process

---

## Concepts

**Confusion matrix**: where the classifier makes mistakes by class.

**Reproducible artifacts**: model file, benchmark reports, quantization summary, evaluation report.

**Receipt**: a small JSON proving all required files were created and basic checks passed.

## Common Pitfalls

* Not running evaluation on held-out test data
* Misinterpreting confusion matrix results
* Forgetting to generate verification artifacts
* Not checking that all pipeline components work together

## Success Criteria

* ✅ `progress/receipt.json` says **PASS**
* ✅ You can explain what each artifact is and where it lives
* ✅ You can describe one change you'd make next (e.g., more data, different architecture)

---

## Setup & Environment Check


# ruff: noqa: E401
import os
import sys
from pathlib import Path

def cd_repo_root():
    p = Path.cwd()
    for _ in range(5):  # klättra uppåt max 5 nivåer
        if (p/"verify.py").exists() and (p/"scripts"/"evaluate_onnx.py").exists():
            if str(p) not in sys.path: sys.path.insert(0, str(p))
            if p != Path.cwd():
                os.chdir(p)
                print("-> Changed working dir to repo root:", os.getcwd())
            return
        p = p.parent
    raise RuntimeError("Could not locate repo root")

cd_repo_root()

# Hints & Solutions helper (pure Jupyter, no extra deps)
from IPython.display import Markdown, display

def hints(*lines, solution: str | None = None, title="Need a nudge?"):
    """Render progressive hints + optional collapsible solution."""
    md = [f"### {title}"]
    for i, txt in enumerate(lines, start=1):
        md.append(f"<details><summary>Hint {i}</summary>\n\n{txt}\n\n</details>")
    if solution:
        # keep code fenced as python for readability
        md.append(
            "<details><summary><b>Show solution</b></summary>\n\n"
            f"```python\n{solution.strip()}\n```\n"
            "</details>"
        )
    display(Markdown("\n\n".join(md)))


## 🤔 Vad är utvärdering och varför behöver vi det?

**Utvärdering** = testa modellen på data den inte har sett under träning.

**Vad vi mäter**:
- **Accuracy** - hur många förutsägelser som är rätta
- **Confusion matrix** - detaljerad breakdown av rätta/felaktiga förutsägelser
- **Per-class performance** - hur bra modellen är på varje klass

**Varför viktigt**:
- **Validering** - säkerställer att modellen faktiskt fungerar
- **Debugging** - visar vilka klasser som är svåra
- **Jämförelse** - kan jämföra olika modeller/inställningar

<details>
<summary>🔍 Klicka för att se vad en confusion matrix visar</summary>

**Confusion matrix**:
- **Diagonal** = rätta förutsägelser
- **Off-diagonal** = felaktiga förutsägelser
- **Per class** = precision, recall för varje klass

</details>


In [ ]:
# Kör utvärdering på vår modell
print("🔍 Kör utvärdering...")

# Använd modellen från föregående notebooks (eller skapa en snabb)
!python -m piedge_edukit.train --fakedata --no-pretrained --epochs 1 --batch-size 256 --output-dir ./models_eval


In [ ]:
# Kör utvärdering med begränsat antal samples (snabbare)
!python scripts/evaluate_onnx.py --model ./models_eval/model.onnx --fakedata --limit 32


In [ ]:
# Visa utvärderingsresultat
import os

if os.path.exists("./reports/eval_summary.txt"):
    with open("./reports/eval_summary.txt", "r") as f:
        print("📊 Utvärderingsresultat:")
        print(f.read())
else:
    print("❌ Utvärderingsrapport missing")


In [ ]:
# Visa träningsgrafer om de finns
from PIL import Image
from IPython.display import display

if os.path.exists("./reports/training_curves.png"):
    print("📈 Träningsgrafer:")
    display(Image.open("./reports/training_curves.png"))
else:
    print("⚠️ Träningsgrafer missing – kör träningen först.")


In [ ]:
# Visa confusion matrix om den finns
import matplotlib.pyplot as plt
from PIL import Image

if os.path.exists("./reports/confusion_matrix.png"):
    print("📈 Confusion Matrix:")
    img = Image.open("./reports/confusion_matrix.png")
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Confusion Matrix')
    plt.show()
else:
    print("❌ Confusion matrix missing")


## 🔍 Automatisk verifiering

**Verifiering** = automatiska checks som säkerställer att lektionen fungerar korrekt.

**Vad kontrolleras**:
- **Artefakter finns** - alla nödvändiga filer är skapade
- **Benchmark fungerar** - latens-data är giltig
- **Kvantisering fungerar** - kvantiserad modell är skapad
- **Utvärdering fungerar** - confusion matrix och accuracy är tillgänglig

**Resultat**: `progress/receipt.json` med PASS/FAIL status


In [ ]:
# Kör automatisk verifiering
print("🔍 Kör automatisk verifiering...")
!python verify.py


In [ ]:
# Analysera kvittot i detalj
import json

if os.path.exists("./progress/receipt.json"):
    with open("./progress/receipt.json", "r") as f:
        receipt = json.load(f)
    
    print("📋 Detaljerad kvitto-analys:")
    print(f"Status: {'✅ PASS' if receipt['pass'] else '❌ FAIL'}")
    print(f"Timestamp: {receipt['timestamp']}")
    
    print("\n🔍 Kontroller:")
    for check in receipt['checks']:
        status = "✅" if check['ok'] else "❌"
        print(f"  {status} {check['name']}: {check['reason']}")
    
    print("\n📊 Metrics:")
    if 'metrics' in receipt:
        for metric, value in receipt['metrics'].items():
            print(f"  {metric}: {value}")
    
    print("\n📁 Genererade filer:")
    if 'artifacts' in receipt:
        for artifact in receipt['artifacts']:
            print(f"  - {artifact}")
else:
    print("❌ Kvitto missing")


## 🤔 Reflektionsfrågor

### TODO R1 — Reflect on results (2–4 bullets)
- Where did quantization help / hurt?
- Do your p50 and p95 match expectations after warm-up?
- One change you would make before deploying.

<details><summary>Hint</summary>
Tie back to goals: correctness, latency, and determinism. Fallback to FP32 is fine if INT8 regresses.
</details>

<details>
<summary>💭 Vilka mål verifieras av vår automatiska check?</summary>

**Svar**: Vår verifiering kontrollerar:
- **Teknisk funktionalitet** - alla steg körs utan fel
- **Artefakt-generering** - nödvändiga filer skapas
- **Data-integritet** - rapporter är giltiga och parseable
- **Pipeline-integration** - alla komponenter fungerar tillsammans

**Vad som INTE verifieras**:
- Accuracy-kvalitet (bara att utvärdering körs)
- Latens-mål (bara att benchmark körs)
- Produktionsredo (bara att pipeline fungerar)

</details>

<details>
<summary>💭 Vad missing för "produktion"?</summary>

**Svar**: För produktion behöver vi:
- **Riktig data** - inte FakeData
- **Accuracy-mål** - specifika krav på precision/recall
- **Latens-mål** - SLA-krav på inference-tid
- **Robusthet** - hantering av edge cases och fel
- **Monitoring** - kontinuerlig övervakning av prestanda
- **A/B-testing** - jämförelse av olika modeller
- **Rollback** - möjlighet att gå tillbaka till tidigare version

</details>


## 🎯 Ditt eget experiment

**Uppgift**: Kör verifiering på olika modeller och jämför kvittona.

**Förslag**:
- Träna modeller med olika inställningar
- Kör verifiering på varje modell
- Jämför kvittona och se vilka som passerar/failar
- Analysera vilka checks som är mest kritiska

**Kod att modifiera**:
```python
# Träna olika modeller och kör verifiering
MODELS = [
    {"epochs": 1, "batch_size": 128, "name": "quick"},
    {"epochs": 3, "batch_size": 64, "name": "balanced"},
    {"epochs": 5, "batch_size": 32, "name": "thorough"}
]

for model_config in MODELS:
    # Träna modell
    # Kör verifiering
    # Analysera kvitto
```


In [ ]:
# TODO: Implementera ditt experiment här
# Träna olika modeller och jämför kvittona

MODELS = [
    {"epochs": 1, "batch_size": 128, "name": "quick"},
    {"epochs": 3, "batch_size": 64, "name": "balanced"},
    {"epochs": 5, "batch_size": 32, "name": "thorough"}
]

print("🧪 Mitt experiment: Jämför olika modeller")
for model_config in MODELS:
    print(f"  - {model_config['name']}: epochs={model_config['epochs']}, batch_size={model_config['batch_size']}")

# TODO: Implementera loop som tränar och verifierar varje modell


## Final Reflection

Congratulations! You've completed the entire PiEdge EduKit lesson. Please reflect on your learning experience:

**1. What was the most challenging part of implementing the CNN architecture? What helped you understand it better?**

*Your answer here (2-3 sentences):*

---

**2. How did your understanding of model performance change after running the latency benchmarks?**

*Your answer here (2-3 sentences):*

---

**3. What surprised you most about the quantization process? What would you do differently in a real deployment?**

*Your answer here (2-3 sentences):*

---

**4. How important do you think automated verification is for ML pipelines? Why?**

*Your answer here (2-3 sentences):*

---

## Next Steps

**Congratulations!** You've successfully completed the PiEdge EduKit lesson. You now understand:

- ✅ CNN implementation and training
- ✅ Model export to ONNX format  
- ✅ Performance benchmarking and analysis
- ✅ Quantization and compression techniques
- ✅ Evaluation and verification workflows

**Real-world applications**: Experiment with real data, different models, or deploy on Raspberry Pi!

**Key concepts mastered**:
- **Training**: Implementing and training neural networks
- **Export**: Converting models to deployment-ready formats
- **Benchmarking**: Measuring and analyzing performance
- **Quantization**: Optimizing models for edge deployment
- **Verification**: Automated quality assurance for ML pipelines
